In [ ]:
%load_ext autoreload
%autoreload 2

## TODOs
- Rendering loop in cpp/rust - also valuable experience :)
- Similarly to how we do binning with alpha, we could do it with bounding boxes to speed up rendering loop (even though we would be looking at some gsns multiple times, it probably pays off)

\#\#\#\#\#

- depth map AND pre-compute alpha? Should be possible -- iterate over x's, y's and depths, check with depth_map (computed by virtue of random sampling) whether to actually perform the computation. Could even make depth vs depth_map part of mask if we're clever about it (e.g. max depth over grids, check if x's and y's of one primitive are contained in those grids)?
- pre-compute alpha while sorting
- replace sorting by top-K iteratively, make sorting "part of rendering" and compute alpha in parallell while sorting... Sorting is becoming a slow part -- check out [Radix Sort](https://www.geeksforgeeks.org/radix-sort/)
- Is batched rendering possible? Maybe we have to make major changes to do so, but that's probably the weakest joint. 
- Further explore: Is it faster to render float16 images? Doing the full calculation in float16 doesn't work, but maybe the rendering is okay? Are other memory improvements possible?
- Bug: depending on `skip` parameter, the rendered pictures look different


# Scribble notebook
The plan is to use this to make first experiments, which will later be turned into a cleaner implementation. 
For now, it is based on https://github.com/thomasantony/splat/blob/master/notes/00_Gaussian_Projection.ipynb 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm 

from utils.Camera import Camera
from utils.util_gau import load_ply, GaussianData
from pathlib import Path 
from decouple import config

In [ ]:
gaussian_objects = load_ply(str(Path(config('MODEL_PATH'))/'debug/point_cloud/iteration_30000/point_cloud.ply'), dtype=np.float64, return_type='gm')


In [ ]:
from utils.rendering import ParRenderer 

(w, h) = (400, 400)

renderer = ParRenderer(gaussian_objects)

camera = Camera(h, w, position=(-1.6, 2., 1.4), target=(-.15, 1.5, .7))# target is pos for one of the gaussians
bitmap_parts = list(tqdm(renderer.plot_model_par_multiprimitives(camera, n_threads=20, skip=16000, K=None)))
# bitmap_parts = list(tqdm(gaussian_objects.get_bm_parts(camera, n_threads=20)))


# do alpha blending post-hoc
bitmap = np.zeros(bitmap_parts[0][0].shape)
alpha = np.zeros(bitmap_parts[0][1].shape)
for bm, al in bitmap_parts:
    tmp = (1-alpha)*al
    bitmap = bitmap + (1-alpha)[..., np.newaxis]*bm
    alpha = alpha + tmp

plt.figure(figsize=(6, 6))
plt.imshow(bitmap, vmin=0, vmax=1.)
plt.show()


## Old Code
Testing out various ideas described in the blog. I might revisit them.

In [ ]:
# def plot_conics_and_bbs(gaussian_objects, camera: Camera, color: str='blue'):
#     # credit to https://github.com/thomasantony/splat/blob/master/notes/00_Gaussian_Projection.ipynb 
#     # Note: there are some things I would still like to change in this function (e.g. I'm pretty sure coordxy does not need four, but rather two, corners)
#     ax = plt.gca()

#     for g in gaussian_objects: #zip(gaussian_objects, colors):
#         assert isinstance(g, Gaussian)
#         (conic, bboxsize_cam, bbox_ndc) = g.get_conic_and_bb(
#             camera, 
#             optimal=True
#         )
#         if conic is None:
#             continue

#         A, B, C = conic
#         # coordxy is the correct scale to be used with gaussian and is already
#         # centered on the gaussian
#         coordxy = bboxsize_cam
#         x_cam = np.linspace(coordxy[0][0], coordxy[1][0], 100)
#         y_cam = np.linspace(coordxy[1][1], coordxy[2][1], 100) # how come the first axis has more than 2 dimensions here?
#         X, Y = np.meshgrid(x_cam, y_cam)
        
#         # 1-sigma ellipse # actually, I think this is the sqrt(3)-sigma ellipse. 
#         # F = A*X**2 + 2*B*X*Y + C*Y**2 - 3.00
#         F = np.sqrt(A*X**2 + 2*B*X*Y + C*Y**2) - 3.00 

#         bbox_screen = camera.ndc_to_pixel(bbox_ndc)

#         # Use bbox offset to position of gaussian in screen coords to position the ellipse
#         x_px = np.linspace(bbox_screen[0][0], bbox_screen[1][0], 100)
#         y_px = np.linspace(bbox_screen[2][1], bbox_screen[1][1], 100) # again, why this many dimensions?
#         X_px, Y_px = np.meshgrid(x_px, y_px)
#         F_val = 0.0
#         plt.contour(X_px, Y_px, F, [F_val])

#         # Plot a rectangle around the gaussian position based on bb
#         ul = bbox_screen[0,:2]
#         ur = bbox_screen[1,:2]
#         lr = bbox_screen[2,:2]
#         ll = bbox_screen[3,:2]
#         ax.add_patch(plt.Rectangle((ul[0], ul[1]), ur[0] - ul[0], lr[1] - ur[1], fill=False, alpha=1., color=color))


In [ ]:
# def plot_opacity_v(gaussian: Gaussian, camera: Camera, bitmap: np.ndarray, alphas: np.ndarray, alpha_thresh: float=None):
#     """Vectorized version of the plot_opacity function of the original repo - we want to loop as little as possible for speed"""
    
    
#     conic, bboxsize_cam, bbox_ndc = gaussian.get_conic_and_bb(camera, optimal=True) # different bounding boxes (active areas for gaussian)
#     # bboxsize_cam, bbox_ndc = gaussian.get_nonopt_bb(camera)
#     h, w = bitmap.shape[:2]
#     bbox_screen = camera.ndc_to_pixel(bbox_ndc, w, h)
    
#     if np.any(np.isnan(bbox_screen)):
#         return

#     ul = bbox_screen[0,:2] # Bounding box vertices 
#     ur = bbox_screen[1,:2]
#     ll = bbox_screen[3,:2]
    
#     y1 = max([int(np.floor(ul[1])), 0])
#     y2 = min([int(np.ceil(ll[1])), bitmap.shape[0]])
    
#     x1 = max([int(np.floor(ul[0])), 0])
#     x2 = min([int(np.ceil(ur[0])), bitmap.shape[1]])
#     nx = x2 - x1
#     ny = y2 - y1
#     if nx <= 0 or ny <= 0: return bitmap, alphas
    
#     # TODO: this was an attempt at checking whether we have rendered enough so that the alpha value in a pixel
#     # is so high that we do not need to keep rendering deeper Gaussians in that pixel. However, the logic below
#     # was slow enough that it did not pay off to implement this
#     # TODO: possibly we could be cleverer when checking alpha - e.g. by using information from the foreground.
#     # I am working on this

#     # midy = y1+ny//2
#     # midx = x1+nx//2
#     # # if alpha_thresh is not None and (alphas[y1:y2,x1:x2] >= alpha_thresh).sum() >= nx*ny/3: 
#     # #     return bitmap, alphas 
#     # if alpha_thresh is not None and min([ # check alpha on bb borders (heuristic)
#     #     # alphas[y1:y2, x1].max(),
#     #     # alphas[y1:y2, x2-1].max(),
#     #     # alphas[y1, x1:x2].max(),
#     #     # alphas[y2-1, x1:x2].max(),
#     #     alphas[midy, x1],
#     #     alphas[midy, x2-1],
#     #     alphas[y1, midx],
#     #     alphas[y2-1, midx],
#     #     alphas[midx,midx]
#     # ]) >= alpha_thresh: 
#     #     return bitmap, alphas
#     # conic = gaussian.get_conic(camera)
    
#     A, B, C = conic # precision matrix is (A, B; B, C)

#     # Extract out inputs for the gaussian
#     coordxy = bboxsize_cam
#     x_cam_1 = coordxy[0][0]   # ul
#     x_cam_2 = coordxy[1][0]   # ur
#     y_cam_1 = coordxy[1][1]   # ur (y)
#     y_cam_2 = coordxy[2][1]   # lr

#     opacity = gaussian.opacity 

#     camera_dir = gaussian.pos - camera.position
#     camera_dir = camera_dir / np.linalg.norm(camera_dir) # normalized camera viewing direction
#     color = gaussian.get_color(camera_dir)
#     y_cam, x_cam = np.meshgrid(np.linspace(y_cam_1, y_cam_2, ny), np.linspace(x_cam_1, x_cam_2, nx), indexing='ij')
#     power = -(A*x_cam**2 + C*y_cam**2)/2.0 - B * x_cam * y_cam
#     # power = np.clip(power, -np.inf, 0.)
#     alpha_ = opacity * np.exp(power)
#     bitmap[y1:y2, x1:x2] = bitmap[y1:y2, x1:x2] + ((1-alphas[y1:y2, x1:x2])*alpha_).reshape(ny, nx, 1) * color[0:3].reshape(1, 1, -1)
#     alphas[y1:y2, x1:x2] = alphas[y1:y2, x1:x2] + (1-alphas[y1:y2, x1:x2]) * alpha_ 
#     return bitmap, alphas 

# def plot_opacity_multi(gaussians: MultiGaussian, camera: Camera, bitmap: np.ndarray, alphas: np.ndarray, alpha_thresh: float=None):
#     """Vectorized version of the plot_opacity function of the original repo - we want to loop as little as possible for speed"""
#     def scale_wh(tnsr: np.ndarray, h: int, w: int):
#         # scale values of tnsr's -1st axis, which has length 2, from -1, 1 to resp. 0, h and 0, w
#         # replace this: np.array([(points_ndc[0] + 1) * width_half, (1.0 - points_ndc[1]) * height_half])
#         tnsr = np.stack([tnsr[:, :, 0]+1, 1-tnsr[:, :, 1]], axis=-1)
#         tnsr = tnsr * np.stack([np.ones(tnsr.shape[:-1])*h, np.ones(tnsr.shape[:-1])*w], axis=-1)
#         return tnsr / 2

#     conic_, bboxsize_cam, bbox_ndc = gaussians.get_conic_and_bb(camera, optimal=True) # different bounding boxes (active areas for gaussian)
#     h, w = bitmap.shape[:2]
#     # bbox_screen = camera.ndc_to_pixel(bbox_ndc, w, h)
#     bbox_screen = scale_wh(bbox_ndc, h, w)
    
#     if np.any(np.isnan(bbox_screen)):
#         return

#     ul = bbox_screen[:, 0,:2] # Bounding box vertices 
#     ur = bbox_screen[:, 1,:2]
#     ll = bbox_screen[:, 3,:2]
    
#     y1_ = np.maximum(np.floor(ul[:, 1]), np.zeros(len(gaussians))).astype(int)
#     x1_ = np.maximum(np.floor(ul[:, 0]), np.zeros(len(gaussians))).astype(int)

#     y2_ = np.minimum(np.ceil(ll[:, 1]), bitmap.shape[0]*np.ones(len(gaussians))).astype(int)
#     x2_ = np.minimum(np.ceil(ur[:, 0]), bitmap.shape[1]*np.ones(len(gaussians))).astype(int)

#     mask = np.logical_and(x2_ > x1_, y2_ > y1_)
#     x1_ = x1_[mask]
#     x2_ = x2_[mask]
#     y1_ = y1_[mask]
#     y2_ = y2_[mask]
#     conic_ = conic_[mask]

#     nx_ = x2_ - x1_
#     ny_ = y2_ - y1_
#     # if nx <= 0 or ny <= 0: return bitmap, alphas

#     # Extract out inputs for the gaussian
#     coordxy = bboxsize_cam[mask]
#     x_cam_1_ = coordxy[:, 0, 0]   # ul
#     x_cam_2_ = coordxy[:, 1, 0]   # ur
#     y_cam_1_ = coordxy[:, 1, 1]   # ur (y)
#     y_cam_2_ = coordxy[:, 2, 1]   # lr

#     opacity_ = gaussians.opacity[mask]

#     # camera_dir = gaussians.pos - camera.position.reshape((1, *camera.position.shape))
#     # camera_dir = camera_dir / np.linalg.norm(camera_dir, axis=-1) # normalized camera viewing direction
#     # color = gaussians.get_color(camera_dir)
#     color_ = gaussians.get_color(None)[mask] # TODO: would be nice if in MultiGaussian we could index with a mask prior to computations like these, rather than doing the computation and then selecting afterwards
    
    
#     for x_cam_1, x_cam_2, y_cam_1, y_cam_2, x1, x2, y1, y2, nx, ny, opacity, color, conic in zip(
#         x_cam_1_, 
#         x_cam_2_, 
#         y_cam_1_, 
#         y_cam_2_, 
#         x1_, 
#         x2_, 
#         y1_, 
#         y2_, 
#         nx_, 
#         ny_, 
#         opacity_, 
#         color_, 
#         conic_
#     ):
#         # if nx <= 0 or ny <= 0: continue
#         A, B, C = conic 
#         y_cam, x_cam = np.meshgrid(np.linspace(y_cam_1, y_cam_2, ny), np.linspace(x_cam_1, x_cam_2, nx), indexing='ij')
#         power = -(A*x_cam**2 + C*y_cam**2)/2.0 - B * x_cam * y_cam
#         # power = np.clip(power, -np.inf, 0.)
#         alpha_ = opacity * np.exp(power)
#         bitmap[y1:y2, x1:x2] = bitmap[y1:y2, x1:x2] + ((1-alphas[y1:y2, x1:x2])*alpha_).reshape(ny, nx, 1) * color[0:3].reshape(1, 1, -1)
#         alphas[y1:y2, x1:x2] = alphas[y1:y2, x1:x2] + (1-alphas[y1:y2, x1:x2]) * alpha_ 
#     return bitmap, alphas 
    
#     # below is WIP code
#     # np.linspace(np.zeros((2,)), np.array([1., 2.]), np.array([10, 20])) is not valid python, but
#     # np.linspace(np.zeros((2,)), np.array([1., 2.]), 10) is
#     n_gaussians = mask.sum()

#     ymin, ymax = np.min(y1_), np.max(y2_)
#     xmin, xmax = np.min(x1_), np.max(x2_)
#     alpha_computations = np.zeros((n_gaussians, ymax - ymin, xmax - xmin))
#     coefs = np.zeros(alpha_computations.shape)
#     side_coefs = np.zeros(alpha_computations.shape)

#     def fill_array(i: int):
#         """Take an index argument to fill the above tensor with values in a vectorized fashion"""
#         A, B, C = conic_[i][0], conic_[i][1], conic_[i][2]
#         y_cam, x_cam = np.meshgrid(np.linspace(y_cam_1_[i], y_cam_2_[i], ny_[i]), np.linspace(x_cam_1_[i], x_cam_2_[i], nx_[i]), indexing='ij')
#         power = -(A*x_cam**2 + C*y_cam**2)/2.0 - B * x_cam * y_cam
#         alpha_computations[i, y1_[i]-ymin:y2_[i]-ymin, x1_[i]-xmin:x2_[i]-xmin] = opacity_[i] * np.exp(power)
#         # return opacity_[i] * np.exp(power)
#     def prod_arr(i: int):
#         # return np.prod(1-alpha_computations[:i+1], axis=0)
#         coefs[i] = np.prod(1-alpha_computations[:i+1], axis=0)
#         # return np.prod(1-alpha_computations[:i+1], axis=0)
#     def other_coefs(i: int):
#         # let a:=alpha_computations
#         # return a[i] + (1-a[i]) * a[i-1] + (1-a[i])(1-a[i-1])*a[i-2] + ...
#         # Letting p[j] := (1-a[j]) * ... * (1-a[0]), then p is coefs, and we therefore return
#         # a[i] * p[i] / p[i] + a[i-1] * p[i] / p[i-1] + ...
#         side_coefs[i] = (coefs[i].reshape(1, *coefs[i].shape) * alpha_computations[:i+1] / coefs[:i+1]).sum(axis=0)
#         # side_coefs[i] = np.stack([coefs[i] / np.maximum(coefs[j], 1e-12) * alpha_computations[j] for j in range(i+1)], axis=0).sum(axis=0)
#         # return np.stack([coefs[i] / np.maximum(coefs[j], 1e-12) * alpha_computations[j] for j in range(i+1)], axis=0).sum(axis=0)
#     np.vectorize(fill_array)(range(n_gaussians))
#     np.vectorize(prod_arr)(range(n_gaussians)) # coefs[i] = (1-a[0]) * ... * (1-a[i])
#     np.vectorize(other_coefs)(range(n_gaussians)) 
#     # for func in [fill_array, prod_arr, other_coefs]:
#     #     for i in range(n_gaussians): func(i)
#     # np.apply_along_axis(fill_array, axis=0, arr=np.arange(n_gaussians, dtype=int).reshape(-1, 1))# (range(n_gaussians))
#     # np.apply_along_axis(prod_arr, axis=0, arr=np.arange(n_gaussians, dtype=int).reshape(-1, 1))# (range(n_gaussians)) # coefs[i] = (1-a[0]) * ... * (1-a[i])
#     # np.apply_along_axis(other_coefs, axis=0, arr=np.arange(n_gaussians, dtype=int).reshape(-1, 1))# (range(n_gaussians)) 

#     new_alphas = side_coefs + coefs * alphas[ymin:ymax, xmin:xmax].reshape(1, *alpha_computations.shape[1:]) 
#     # TODO: below, do I have to multiply by something more too?
#     new_colors = bitmap[ymin:ymax, xmin:xmax].reshape(1, *alpha_computations.shape[1:], 3) + new_alphas.reshape(*new_alphas.shape, 1) * color_[:, :3].reshape(-1, 1, 1, 3)
#     bitmap[ymin:ymax, xmin:xmax] = new_colors.sum(axis=0)
#     alphas[ymin:ymax, xmin:xmax] = new_alphas[-1]
#     return bitmap, alphas 

# # # Plot some Gaussians
# # loc_scaler = 1
# # scale_scaler = .05
# # rot_scaler = 10
# # n_gaussians = 50
# # loc_bias = 0. # np.ones(3,) * -.1

# # gaussians_debug = [
# #     Gaussian((np.random.rand(3, )-.5 + loc_bias)*2*loc_scaler, np.random.rand(3)*scale_scaler, np.random.rand(4)*rot_scaler, np.array([1.]), np.array([ 1.772484, -1.772484,  1.772484])) for _ in range(n_gaussians)
# # ]
# # gaussian_objects = gaussians_debug

# # (h, w) = (300, 400)
# # camera = Camera(h, w)
# # # Get gaussian indices sorted by depth
# # indices = np.argsort([g.get_depth(camera) for g in gaussian_objects]) # TODO: vectorize get_depth

# # # Initialize a bitmap with alpha channel of size w x h

# # bitmap = np.zeros((h, w, 3), np.float32)
# # alphas = np.zeros((h, w), np.float32)

# # plt.figure(figsize=(6,6))
# # for idx in indices:
# #     # bitmap, alphas = plot_opacity(gaussian_objects[idx], camera, bitmap, alphas)
# #     bitmap, alphas = plot_opacity_v(gaussian_objects[idx], camera, bitmap, alphas)
# # print(f'after execution, {bitmap.max()=}')
# # # Plot the bitmap
# # plt.imshow(bitmap, vmin=0, vmax=1.0)

# # plt.show()

In [ ]:
# Old classes for plotting which don't use MultiGaussian

# def helper_primitives(indices: List[int], camera, bitmap, alphas, alpha_thresh):
#     for idx in indices:
#         bitmap, alphas = plot_opacity_v(gaussian_objects[idx], camera, bitmap, alphas, alpha_thresh)
#     return bitmap, alphas 

# def plot_model_par_primitives(camera, gaussian_objects: List[Gaussian], alpha_thresh: float=None, n_threads:int=1):
#     """Parallellize image rendering by splitting the sorted list of Gaussians and rendering them individually.
#     Requires blending the segments later (for now).
#     """
#     print('Sorting the gaussians by depth')
#     indices = np.argsort([gau.get_depth(camera) for gau in gaussian_objects])# [::-1] # fast-ish. probably get_depth is slowing it down (vectorization/parallellization TODO)
#     w, h = camera.w, camera.h
    
#     print('Plotting with', len(gaussian_objects), 'gaussians')
#     bitmap = np.zeros((h, w, 3), np.float32)
#     alphas = np.zeros((h, w), np.float32)
#     skip = len(indices) // (3*n_threads) # arbitrary choice TODO
#     gsns = [indices[i*skip:(i+1)*skip] for i in range(len(indices)//skip + int(len(indices)%skip != 0))]
#     with mp.Pool(n_threads) as pool:
#         for r in pool.imap(
#             functools.partial(
#                 helper_primitives,
#                 alpha_thresh=alpha_thresh, 
#                 camera=camera,# None,
#                 bitmap=bitmap,
#                 alphas=alphas
#             ), 
#             gsns
#         ): 
#             yield r 

In [ ]:
# # Test IterativeImageSegmenter
# # The idea is to segment an image such that each segment has roughly the same number of Gaussians.
# # It's still not perfect (see blog)
# # TODO: There seems to be a bug that causes some of the splits to be sub-optimal. Have to investigate
# loc_scaler = 1
# scale_scaler = .03
# rot_scaler = 10
# n_gaussians = 30
# loc_bias = 0. 

# gaussians_debug = [
#     Gaussian((np.random.rand(3, )-.5 + loc_bias)*2*loc_scaler, np.random.rand(3)*scale_scaler, np.random.rand(4)*rot_scaler, np.array([1.]), np.array([ 1.772484, -1.772484,  1.772484])) for _ in range(n_gaussians)
# ]

# fig = plt.figure()
# ax = plt.gca()
# camera = Camera(100, 100)
# plot_conics_and_bbs(gaussians_debug, camera)
# image = SubImage(np.zeros((100, 100)), (0, 0), (100, 100), camera)
# pset = PrimitiveSet(gaussians_debug)
# sset = PrimitiveSubset(pset, list(range(len(gaussians_debug))))
# segmenter = IterativeImageSegmenter(sset, image, camera, thresh=3)

# for i in range(7):
#     bx, by = segmenter.cut(i)
#     ul, lr = segmenter.cuts[-1]['corners']
#     ymin, xmin = ul
#     ymax, xmax = lr 
#     if bx is None:
#         # plot y splitting line
#         plt.plot([xmin, xmax], [by]*2)
#         plt.text((xmax+xmin)/3*2, by, str(i))
#     else:
#         plt.plot([bx]*2, [ymin, ymax])
#         plt.text(bx, (ymin+ymax)/3*2, str(i))
    
# plt.xlim([0, camera.w])
# plt.ylim([0, camera.h])
# plt.grid(True)
# plt.show()
